In [17]:
import math
import pygame
import random
import numpy as np
#from tf_transformations import euler_from_quaternion

NUM_BOIDS = 3

# Colors
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ARENA_BLUE = (11, 11, 255)

LED_COLOURS = [RED, GREEN, BLUE]

# Screen dimensions (in pixels)
ARENA_WIDTH = 1920
ARENA_HEIGHT = 1080

BUFFER_SPACE = 200
V_UPPER = ARENA_HEIGHT - BUFFER_SPACE
V_LOWER = BUFFER_SPACE

H_UPPER = ARENA_WIDTH - BUFFER_SPACE
H_LOWER = BUFFER_SPACE

ROBOT_RADIUS =100

NUM_LEDS = 16

# Boid parameters
MAX_SPEED = 2
VIEW_DISTANCE = 1000
SEPARATION_DISTANCE = ROBOT_RADIUS*4
ALIGNMENT_FACTOR = 0.01
COHESION_FACTOR = 0.02
SEPARATION_FACTOR = 1.50

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((ARENA_WIDTH, ARENA_HEIGHT))
clock = pygame.time.Clock()


# Boid class
class Boid:
    def __init__(self):
        self.x = random.randint(H_LOWER, H_UPPER)
        self.y = random.randint(V_LOWER, V_UPPER)
        self.angle = random.uniform(0, 2*math.pi)
        self.velocity = 1
        self.LED = WHITE

    def move(self):
        self.x += self.velocity * math.cos(self.angle)
        self.y += self.velocity * math.sin(self.angle)

    def draw(self):
        pygame.draw.circle(screen, BLACK, (int(self.x), int(self.y)), ROBOT_RADIUS)
        pygame.draw.line(screen, WHITE, (int(self.x+ROBOT_RADIUS*3/4*math.cos(self.angle)), int(self.y+ROBOT_RADIUS*3/4*math.sin(self.angle))), (int(self.x+ROBOT_RADIUS*math.cos(self.angle)), int(self.y+ROBOT_RADIUS*math.sin(self.angle))), width = 3)
        
        pygame.draw.circle(screen, WHITE, (int(self.x-ROBOT_RADIUS*math.sin(self.angle)), int(self.y+ROBOT_RADIUS*math.cos(self.angle))), ROBOT_RADIUS/20)
        pygame.draw.circle(screen, WHITE, (int(self.x+ROBOT_RADIUS*math.sin(self.angle)), int(self.y-ROBOT_RADIUS*math.cos(self.angle))), ROBOT_RADIUS/20)
        
        for i in range(0, NUM_LEDS):
            pygame.draw.circle(screen, self.LED, (int(self.x+ROBOT_RADIUS/2*math.cos(self.angle+i*(2*math.pi/NUM_LEDS))), int(self.y+ROBOT_RADIUS/2*math.sin(self.angle+i*(2*math.pi/NUM_LEDS)))), 1)


# Function to implement Boid rules
def next_step(main_boid, other_boids):
    
    aggregate_cohesion = Vec2(0.0, 0.0)
    aggregate_alignment = Vec2(0.0, 0.0)
    aggregate_separation = Vec2(0.0, 0.0)
    
    count_cohesion = 0.0
    count_alignment = 0.0
    count_separation  = 0.0
    
    main_pos = Vec2.from_pose(main_boid)
    for other_boid in other_boids:
        other_pos = Vec2.from_pose(other_boid) 
        
        
        diff = other_pos.sub(main_pos)
        dist = diff.length()
        
        if dist <= VIEW_DISTANCE:
            # Cohesion
            norm = diff.normalised()
            aggregate_cohesion= aggregate_cohesion.add(norm)
            count_cohesion += 1
            
            # Alignment
            other_dir = Vec2.from_angle(other_boid.angle)
            aggregate_alignment = aggregate_alignment.add(other_dir.normalised())
            count_alignment += 1
            
            # Separation
            if (dist <= SEPARATION_DISTANCE):
                aggregate_separation = aggregate_separation.add(norm.inversed())
                count_separation += 1
         
    aggregate_cohesion = aggregate_cohesion.divide(count_cohesion)
    aggregate_alignment = aggregate_alignment.divide(count_alignment)
    aggregate_separation = aggregate_separation.divide(count_separation)
    
    aggregate_cohesion = aggregate_cohesion.scale(COHESION_FACTOR)
    aggregate_alignment = aggregate_alignment.scale(ALIGNMENT_FACTOR)
    aggregate_separation = aggregate_separation.scale(SEPARATION_FACTOR)
      
    led_decision = np.argmax([Vec2.length(aggregate_separation), Vec2.length(aggregate_cohesion), Vec2.length(aggregate_alignment)])
    main_boid.LED = LED_COLOURS[led_decision]
    if (count_cohesion == 0):
        main_boid.LED = WHITE

    boundary = Vec2(0.0, 0.0)
    
    if ((main_pos.x < H_LOWER) or
        (main_pos.x > H_UPPER) or
        (main_pos.y < V_LOWER) or 
        (main_pos.y > V_UPPER)):
        boundary = main_pos.normalised().inversed()
    
    
    direction = boundary.add(aggregate_cohesion).add(aggregate_alignment).add(aggregate_separation)
    delta  = direction.angle() * min(direction.length(), 0.05)
    
    # Return (Linear Velocity Forward, Angular Velocity Yaw)
    main_boid.velocity = MAX_SPEED
    main_boid.angle += delta


    
class Vec2():
    def __init__(self, x_ = 0.0, y_ = 0.0):
        self.x = x_
        self.y = y_
        
    def from_pose(pose):
        return Vec2(pose.x, pose.y)
    
    def from_angle(radians):
        return Vec2( math.cos(radians), math.sin(radians) )
    
    
    
    def sub(self, other):
        return Vec2(self.x - other.x, self.y - other.y)
    
    def add(self, other):
        return Vec2(self.x + other.x, self.y + other.y)
    
    def scale(self, scale):
        return Vec2(self.x * scale, self.y * scale)
    
    def divide(self, denom):
        if (denom == 0.0):
            return Vec2(0.0, 0.0)
        return self.scale(1.0 / denom)
    
    def inversed(self):
        return self.scale(-1.0)
    
    def length(self):
        return math.sqrt((self.x ** 2) + (self.y ** 2))
    
    def normalised(self):
        return self.divide(self.length())
    
    def angle(self):
        return math.atan2(self.x, self.y)

    
# Create boids
boids = [Boid() for _ in range(NUM_BOIDS)]
running = True
while running:
    screen.fill(ARENA_BLUE)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    
    for boid in boids:
        
        other_boids = []
        
        for boid_test in boids:
            if boid_test != boid:
                other_boids.append(boid_test)
            
        next_step(boid, other_boids)
        boid.move() 
        boid.draw()
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()